<a href="https://colab.research.google.com/github/m-kafaiekhou/LLM-fine-tuning/blob/main/PersianMind_TU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

# Load PersianMind model and tokenizer
model_id = "universitytehran/PersianMind-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")  # Use GPU if available

# Function to get streaming responses
def chat_with_model(prompt, max_new_tokens=200, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move to GPU
    streamer = TextStreamer(tokenizer)  # Enables streaming output

    # Generate response
    with torch.no_grad():
        model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=1.1,  # Helps reduce repetitive responses
            streamer=streamer
        )

# # Example interactive chat loop
# while True:
#     user_input = input("📝 You: ")  # Get user input
#     if user_input.lower() in ["exit", "quit", "bye"]:
#         print("👋 Exiting chat...")
#         break

#     print("🤖 PersianMind: ", end="")  # Show bot response prefix
#     chat_with_model(user_input)  # Generate streaming response
#     print("\n")  # Newline for better formatting


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/688k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.50M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

In [8]:
TEMPLATE = "{context}\nYou: {prompt}\nPersianMind: "
CONTEXT = "This is a conversation with PersianMind. It is an artificial intelligence model designed by a team of " \
    "NLP experts at the University of Tehran to help you with various tasks such as answering questions, " \
    "providing recommendations, and helping with decision making. You can ask it anything you want and " \
    "it will do its best to give you accurate and relevant information."
PROMPT = "راجع به سیستم های کنترلی در پتروشیمی چی میدونی؟"

model_input = TEMPLATE.format(context=CONTEXT, prompt=PROMPT)
input_tokens = tokenizer(model_input, return_tensors="pt")
input_tokens = input_tokens.to("cuda")
generate_ids = model.generate(**input_tokens, max_new_tokens=512, do_sample=False, repetition_penalty=1.1)
model_output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print(model_output[len(model_input):])

 سیستم های کنترلی در پتروشیمی، به طور کلی به دو دسته تقسیم می شوند: 1) سیستم های کنترل مستقیم 2) سیستم های کنترل غیر مستقیم 3) سیستم های کنترل ترکیبی 4) سیستم های کنترل هوشمند 5) سیستم های کنترل خودکار 6) سیستم های کنترل مبتنی بر هوش مصنوعی 7) سیستم های کنترل مبتنی بر یادگیری ماشین 8) سیستم های کنترل مبتنی بر تشخیص الگو 9) سیستم های کنترل مبتنی بر تشخیص الگو 10) سیستم های کنترل مبتنی بر تشخیص الگو 11) سیستم های کنترل مبتنی بر تشخیص الگو 12) سیستم های کنترل مبتنی بر تشخیص الگو 13) سیستم های کنترل مبتنی بر تشخیص الگو 14) سیستم های کنترل مبتنی بر تشخیص الگو 15) سیستم های کنترل مبتنی بر تشخیص الگو 16) سیستم های کنترل مبتنی بر تشخیص الگو 17) سیستم های کنترل مبتنی بر تشخیص الگو 18) سیستم های کنترل مبتنی بر تشخیص الگو 19) سیستم های کنترل مبتنی بر تشخیص الگو 20) سیستم های کنترل مبتنی بر تشخیص الگو 21) سیستم های کنترل مبتنی بر تشخیص الگو 22) سیستم های کنترل مبتنی بر تشخیص الگو 23) سیستم های کنترل مبتنی بر تشخیص الگو 24) سیستم های کنترل مبتنی بر تشخیص الگو 25) سیستم های کنترل مبتنی بر تشخیص الگو